In [1]:
import Pkg
#Pkg.add("JuMP")
#Pkg.add("GLPK")
#Pkg.add("Gruobi")
#Pkg.add("DataFrames")
#Pkg.add("CSV")
#Pkg.add("PyCall")

In [2]:
using JuMP, GLPK
using DataFrames
using CSV

In [3]:
# Load the data
scenarios_200 = CSV.read("../data/scenarios_200.csv", DataFrame)

# create a dictonary with 200 dataframes for each scenario
scenarios = Dict()
for i in 1:200
    df_helper = DataFrame(scenarios_200[:,3*i-2:3*i])
    #set column names
    rename!(df_helper, [:"price", :"wind power", :"grid_excess"])
    scenarios[i] = df_helper
end

In [4]:
scenarios[1]

Row,price,wind power,grid_excess
,Float64,Float64,Int64
1,122.83,151.992,1
2,132.91,143.459,1
3,132.9,142.587,0
4,125.03,146.618,1
5,120.07,150.136,1
6,139.83,151.102,1
7,164.23,145.927,0
8,186.66,143.261,1
9,186.95,142.279,0


In [5]:
scenarios[2]

Row,price,wind power,grid_excess
,Float64,Float64,Int64
1,2.01,22.3372,1
2,1.38,18.6099,1
3,0.09,15.9499,0
4,0.08,13.4696,1
5,0.05,9.2429,1
6,0.0,5.90827,1
7,-0.38,4.51948,0
8,0.0,2.73391,1
9,0.0,1.1039,0


In [6]:
# Initialize the DataFrame directly without dynamic column names
result_df = DataFrame(hour = Int[], p_DA = Float64[])

Row,hour,p_DA
,Int64,Float64


In [7]:
W = 200
hours = 24

24

In [8]:
# Create a new model with GLPK solver
model = Model(GLPK.Optimizer)

# Define the decision variables for hour
@variable(model, p_DA[1:hours])
@variable(model, delta[1:W,1:hours])
@variable(model, delta_up[1:W,1:hours])
@variable(model, delta_down[1:W,1:hours])

# Define the objective function
@objective(model, Max, sum(1/W*(scenarios[i][hour,"price"] * p_DA[hour] + 0.9*scenarios[i][hour,"price"]*delta_up[i,hour] - 1.2*scenarios[i][hour,"price"]*delta_down[i,hour]) for i in 1:W, hour in 1:hours))

# Define the constraints
@constraint(model, [hour in 1:hours], p_DA[hour] <= 200)
@constraint(model, [hour in 1:hours], p_DA[hour] >= 0)
@constraint(model, [i in 1:W, hour in 1:hours], delta[i,hour] == scenarios[i][hour,"wind power"] - p_DA[hour])
@constraint(model, [i in 1:W, hour in 1:hours], delta[i,hour] == delta_up[i,hour] - delta_down[i,hour])
@constraint(model, [i in 1:W, hour in 1:hours], delta_down[i,hour] >= 0)
#@constraint(model, [i in 1:W, hour in 1:hours], delta_down[i,hour] <= p_DA[hour])
@constraint(model, [i in 1:W, hour in 1:hours], delta_up[i,hour] >= 0)
@constraint(model, [i in 1:W, hour in 1:hours], delta_up[i,hour] + p_DA[hour] <= 200)



# Solve the optimization problem
optimize!(model)


#println(model)

# Print the termination status
status = termination_status(model)
if status == MOI.OPTIMAL
    println("Optimal solution found")
    
    # RETURN OBJECTIVE value
    println("Objective value: ", objective_value(model))
else
    println("No optimal solution found")
end

Optimal solution found
Objective value: 126590.28129513426


In [9]:
println("Objective value: ", objective_value(model))
println("p_DA: ", value.(p_DA))
println("delta: ", value.(delta))
println("delta_up: ", value.(delta_up))
println("delta_down: ", value.(delta_down))

Objective value: 126590.28129513426
p_DA: [41.843387096774194, 37.18483869354839, 35.290483854838705, 29.92041925806452, 29.417048322580648, 26.704693532258062, 34.998064516129034, 35.350903177419355, 25.43998372580645, 25.728967661290326, 29.257467564516134, 29.19587088709677, 31.342757951612903, 28.317257903225805, 24.97045146774193, 28.99003225806452, 31.010870838709675, 32.208774193548386, 32.1540160483871, 34.54677417741936, 33.92741935483871, 31.87990316129032, 35.006960933786075, 32.08983864516129]
delta: [110.14903222580645 106.27435485483869 107.2962903387097 116.69796782258064 120.71891937096771 124.3969193064516 110.92887093548387 107.91022583870969 116.8387259032258 116.48635485483872 112.32124206451613 109.12187103225806 105.99240332258066 108.48919369354839 108.02664527419358 100.53916120967743 90.81912911290323 82.91719348387096 76.34743549999999 68.99419356451611 64.25483869354838 60.286064516129045 49.64578096943973 45.97967746774194; -19.506145258064514 -18.5749678225